In [ ]:
import os
os.environ["GROQ_API_KEY"] = "x"
os.environ["TAVILY_API_KEY"] = "x"

!pip install langchain-groq langchain-community langchain-core langchainhub requests duckduckgo-search


In [2]:
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool

# Create the base LLM
llm = ChatGroq(model="llama3-8b-8192")

# Search Tool for Agent 1
search_tool = TavilySearchResults(api_key="TAVILY_API_KEY")

# Test the tool once
print(search_tool.invoke("latest news on India space mission"))


[{'url': 'https://www.isro.gov.in/', 'content': '**Flash News:**\n[PSLV C60/SPADEX Mission : SpaDeX Deployed! Successful separation of SpaDeX satellites marks another milestone in India\'s space journey.](mission_SpaDeX.html "SpaDeX Deployed! Successful separation of SpaDeX satellites marks another milestone in India\'s space journey.")\n\n**Flash News:**\n[Global Space Exploration Conference (GLEX) 2025 - 7th-9th, May 2025](/media_isro/pdf/Highlights/GLEX_2025_Brochure.pdf "GLEX 2025")\n\n**Flash News:**\n\nOur Instagram  isro.dos [...] **Flash News:**\n[SPADEX UNDOCKING SUCCESSFUL](spadex_undocking_successful.html "SPADEX UNDOCKING SUCCESSFUL")\n\n**Flash News:**\nYuvika-2025: Start of Registration\n\n**Flash News:**\n[100th launch from Sriharikota : The GSLV-F15/NVS-02 mission has been successfully accomplished. India reaches new heights in space navigation!](GSLV-F15_NVS-02_Mission.html "100th launch from Sriharikota : The GSLV-F15/NVS-02 mission has been successfully accomplished.

In [3]:
from langchain import hub
!pip install langchainhub

# Pull ReAct agent prompt
prompt = hub.pull("hwchase17/react")

c:\Users\LENOVO\anaconda3\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [4]:
from langchain.agents import create_react_agent, AgentExecutor

# Agent 1 uses Tavily search to summarize topic
agent1 = create_react_agent(llm=llm, tools=[search_tool], prompt=prompt)
agent_executor1 = AgentExecutor(agent=agent1, tools=[search_tool], verbose=True, handle_parsing_errors=True)


In [5]:
@tool
def analyze_critically(text: str) -> str:
    """Provides critical analysis of the given information."""
    analysis_prompt = f"Critically analyze this news summary and provide possible implications, risks, or biases:\n\n{text}"
    return llm.invoke(analysis_prompt)

# Agent 2 uses the critical thinking tool
agent2 = create_react_agent(llm=llm, tools=[analyze_critically], prompt=prompt)
agent_executor2 = AgentExecutor(agent=agent2, tools=[analyze_critically], verbose=True, handle_parsing_errors=True)


In [6]:
# Input query
query = "India's space mission updates"

# Step 1: Agent 1 summarizes topic from the web
summary_result = agent_executor1.invoke({"input": f"Summarize: {query}"})
summary = summary_result["output"]

print("\n🔍 Agent 1 Summary:")
print(summary)

# Step 2: Agent 2 critically analyzes the summary
analysis_result = agent_executor2.invoke({"input": summary})

print("\n🧠 Agent 2 Critical Analysis:")
print(analysis_result["output"])




> Entering new AgentExecutor chain...
Thought: I need to stay up-to-date on India's space mission updates, which are crucial for understanding the country's advancements in space exploration and technology.

Action: tavily_search_results_json
Action Input: "India's space mission updates"[{'url': 'https://www.pib.gov.in/PressReleasePage.aspx?PRID=2127312', 'content': 'Disclosing this while speaking at a media interaction at National Media Centre here today, Union Minister of State (Independent Charge) for Science and Technology; Earth Sciences and Minister of State for PMO, Department of Atomic Energy, Department of Space, Personnel, Public Grievances and Pensions, Dr. Jitendra Singh provided detailed updates on the mission’s current status, technological progress, and the broader economic dividends emerging from India’s cost-effective space programme. [...] Disclosing this while speaking at a media interaction at National Media Centre here today, Union Minister of State (Independent 